In [1]:
from langchain_community.document_loaders import WebBaseLoader
loader=WebBaseLoader("https://python.langchain.com/docs/tutorials/llm_chain/")
loader

USER_AGENT environment variable not set, consider setting it to identify your requests.


## Load the web page as a document

In [2]:
document=loader.load()
document

[Document(metadata={'source': 'https://python.langchain.com/docs/tutorials/llm_chain/', 'title': 'Build a simple LLM application with chat models and prompt templates | 🦜️🔗 LangChain', 'description': "In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!", 'language': 'en'}, page_content='\n\n\n\n\nBuild a simple LLM application with chat models and prompt templates | 🦜️🔗 LangChain\n\n\n\n\n\n\nSkip to main contentJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!IntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuil

## Split the document as chunks of text

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(document)
documents

[Document(metadata={'source': 'https://python.langchain.com/docs/tutorials/llm_chain/', 'title': 'Build a simple LLM application with chat models and prompt templates | 🦜️🔗 LangChain', 'description': "In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!", 'language': 'en'}, page_content='Build a simple LLM application with chat models and prompt templates | 🦜️🔗 LangChain'),
 Document(metadata={'source': 'https://python.langchain.com/docs/tutorials/llm_chain/', 'title': 'Build a simple LLM application with chat models and prompt templates | 🦜️🔗 LangChain', 'description': "In this quickstart we'll show you how to build a simple LLM application with LangChain.

### Convert the documents to vectors and store in vector DB using Embeddings

In [5]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

embeddings=OpenAIEmbeddings()
vectore_store=FAISS.from_documents(documents=documents,embedding=embeddings)
vectore_store

### Query/Retrieve the vector store

In [6]:
query="This application will translate text from English into another Language"

result=vectore_store.similarity_search(query)
result[0].page_content

'language: The language to translate text into\ntext: The text to translate'

In [7]:
query="Right now we are passing a list of messages directly"

result=vectore_store.similarity_search(query)
result[0].page_content

'messages of the same typeHow to add message historyHow to migrate from legacy LangChain agents to LangGraphHow to retrieve using multiple vectors per documentHow to pass multimodal data directly to modelsHow to use multimodal promptsHow to create a custom Output ParserHow to use the output-fixing parserHow to parse JSON outputHow to retry when a parsing error occursHow to parse text from message objectsHow to parse XML outputHow to parse YAML outputHow to use the Parent Document RetrieverHow to use LangChain with different Pydantic versionsHow to add chat historyHow to get a RAG application to add citationsHow to do per-user retrievalHow to get your RAG application to return sourcesHow to stream results from your RAG applicationHow to split JSON dataHow to recursively split text by charactersResponse metadataHow to pass runtime secrets to runnablesHow to do "self-querying" retrievalHow to split text based on semantic similarityHow to chain runnablesHow to save and load LangChain'

In [8]:
query="play around with this prompt template by itself"

result=vectore_store.similarity_search(query)
result[0].page_content

'for token in model.stream(messages):    print(token.content, end="|")\n|C|iao|!||\nYou can find more details on streaming chat model outputs in this guide.\nPrompt Templates\u200b\nRight now we are passing a list of messages directly into the language model. Where does this list of messages come from? Usually, it is constructed from a combination of user input and application logic. This application logic usually takes the raw user input and transforms it into a list of messages ready to pass to the language model. Common transformations include adding a system message or formatting a template with the user input.\nPrompt templates are a concept in LangChain designed to assist with this transformation. They take in raw user input and return data (a prompt) that is ready to pass into a language model.\nLet\'s create a prompt template here. It will take in two user variables:'

### Creating the Prompt Template

In [10]:
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>
"""
)


### Create a LLM Chatbot Model

In [9]:
from langchain_openai import ChatOpenAI

llm=ChatOpenAI(model="gpt-4o")
print(llm)

client=<openai.resources.chat.completions.Completions object at 0x000001FCAB26D040> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001FCAB26E6F0> root_client=<openai.OpenAI object at 0x000001FC66CB3050> root_async_client=<openai.AsyncOpenAI object at 0x000001FCAB26D0A0> model_name='gpt-4o' model_kwargs={} openai_api_key=SecretStr('**********')


### Create Document chain to retrieve the output

In [11]:
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain=create_stuff_documents_chain(llm,prompt)
document_chain


RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001FCAB26D040>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001FCAB26E6F0>, root_client=<openai.OpenAI object at 0x000001FC66CB3050>, root_async_client=<openai.AsyncOpenAI object at 0x000001FCAB26D0A0>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_docume